# Basic time series exploration

In [1]:
import numpy as np
import pandas as pd

full_data = pd.read_csv("my_expenses.csv", delimiter = ";", decimal = ",")
full_data.head()

,original,Date,amount_kn,Amount,Note,Currency,Category
0,220 kn interšpar,2021-09-25,220.0,29.20,interšpar,EUR,shopping
1,200kn gym,2021-10-01,200.0,26.54,gym,EUR,gym
2,230kn ducan interspar,2021-10-02,230.0,30.53,ducan interspar,EUR,shopping
3,400 kn lijekovi baka + plivit total,2021-10-04,400.0,53.09,lijekovi baka + plivit total,EUR,baka
4,250 kn lidl,2021-10-08,250.0,33.18,lidl,EUR,shopping


In [2]:
full_data_amounts = full_data.drop(["original", "amount_kn", "Note", "Currency"], axis=1)
full_data_amounts.head()

,Date,Amount,Category
0,2021-09-25,29.20,shopping
1,2021-10-01,26.54,gym
2,2021-10-02,30.53,shopping
3,2021-10-04,53.09,baka
4,2021-10-08,33.18,shopping


In [3]:
daily = full_data_amounts.groupby("Date")[["Amount"]].sum()
daily.head()

,Amount
Date,
2021-09-25,29.20
2021-10-01,26.54
2021-10-02,30.53
2021-10-04,53.09
2021-10-07,39.95


In [4]:
daily.index = pd.to_datetime(daily.index)
daily.index

DatetimeIndex(['2021-09-25', '2021-10-01', '2021-10-02', '2021-10-04',
               '2021-10-07', '2021-10-08', '2021-10-09', '2021-10-12',
               '2021-10-16', '2021-10-20',
               ...
               '2024-03-19', '2024-03-20', '2024-03-21', '2024-03-22',
               '2024-03-23', '2024-03-25', '2024-03-26', '2024-03-27',
               '2024-03-28', '2024-03-29'],
              dtype='datetime64[ns]', name='Date', length=502, freq=None)

In [15]:
# Get monthly amounts
monthly = daily.resample("1M").sum()
monthly.tail()

,Amount
Date,
2023-11-30,2373.00
2023-12-31,937.80
2024-01-31,1235.74
2024-02-29,789.50
2024-03-31,952.70


## Creating a Prophet model

In [6]:
# The dataframe should have specific column names
monthly.columns = ["y"]
monthly["ds"] = monthly.index

monthly.reset_index(drop = True)
monthly.head()

,y,ds
Date,,
2021-09-30,29.20,2021-09-30
2021-10-31,659.76,2021-10-31
2021-11-30,840.82,2021-11-30
2021-12-31,378.26,2021-12-31
2022-01-31,825.94,2022-01-31


In [7]:
from prophet import Prophet
model = Prophet()

model.fit(monthly)

17:34:59 - cmdstanpy - INFO - Chain [1] start processing
17:34:59 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
# Predict next year
future = model.make_future_dataframe(periods = 12, freq = "M")
model.predict(future)[["ds", "yhat"]].tail(13)

,ds,yhat
30,2024-03-31,996.433684
31,2024-04-30,1452.682474
32,2024-05-31,1829.970995
33,2024-06-30,212.333125
34,2024-07-31,1839.992002
35,2024-08-31,2528.875465
36,2024-09-30,933.660863
37,2024-10-31,1882.374437
38,2024-11-30,1142.454537
39,2024-12-31,1689.826105
